In [6]:
import pandas as pd
import numpy as np
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM, Dropout
from keras.layers import Dense

# Reading Data

In [7]:
df = pd.read_excel('PrivatizedDataforParticipants.xlsx')
new_header = df.iloc[1] #grab the first row for the header
df = df[2:] #take the data less the header row
df.columns = new_header #set the header row as the df header
df2 = df.drop(['Generic Group','Generic Brand','Generic Product Category','Generic Sub-Variable','Units'],axis=1)
df2 = df2.rename(columns={'Generic LookupKey': 'Generic_LookupKey', 'Generic Variable': 'Generic_Variable'})
df2.reset_index(inplace = True, drop = True)

df2 = df2[df2.Generic_Variable == 'Net Migrations(Norm)']
df2.head()

1,Generic Product,Generic Group variable,Generic_Variable,Generic_LookupKey,2016-04-01 00:00:00,2016-05-01 00:00:00,2016-06-01 00:00:00,2016-07-01 00:00:00,2016-08-01 00:00:00,2016-09-01 00:00:00,...,2019-06-01 00:00:00,2019-07-01 00:00:00,2019-08-01 00:00:00,2019-09-01 00:00:00,2019-10-01 00:00:00,2019-11-01 00:00:00,2019-12-01 00:00:00,2020-01-01 00:00:00,2020-02-01 00:00:00,2020-03-01 00:00:00
13,Panther,Volume,Net Migrations(Norm),Segment 1 - Sandesh Brand 2Sandesh Brand 2Mobi...,1.47449,1.46596,0.511313,0.688626,0.473906,0.331906,...,0.676655,-0.121625,0.761047,0.567425,x,x,x,x,x,x
16,Leopard,Volume,Net Migrations(Norm),Segment 1 - Sandesh Brand 2Sandesh Brand 2Mobi...,-1.51781,-1.4995,0.549954,0.169296,0.630261,0.935109,...,0.161265,2.13523,-0.154505,0.146809,x,x,x,x,x,x
19,Lion,Volume,Net Migrations(Norm),Segment 1 - Sandesh Brand 2Sandesh Brand 2Mobi...,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.11952,-0.989891,-0.92848,-0.430021,x,x,x,x,x,x


## Helper Functions

In [8]:
# split a univariate sequence
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

def forecast_accuracy(forecast, actual):
    mape = np.mean(np.abs(forecast - actual)/np.abs(actual))  # MAPE
    me = np.mean(forecast - actual)             # ME
    mae = np.mean(np.abs(forecast - actual))    # MAE
    mpe = np.mean((forecast - actual)/actual)   # MPE
    rmse = np.mean((forecast - actual)**2)**.5  # RMSE
    return({'mape':mape, 'me':me, 'mae': mae, 
            'mpe': mpe, 'rmse':rmse})

## Net Migrations Panther

In [9]:
df3 = df2.iloc[0:1,:]
df3.head()

1,Generic Product,Generic Group variable,Generic_Variable,Generic_LookupKey,2016-04-01 00:00:00,2016-05-01 00:00:00,2016-06-01 00:00:00,2016-07-01 00:00:00,2016-08-01 00:00:00,2016-09-01 00:00:00,...,2019-06-01 00:00:00,2019-07-01 00:00:00,2019-08-01 00:00:00,2019-09-01 00:00:00,2019-10-01 00:00:00,2019-11-01 00:00:00,2019-12-01 00:00:00,2020-01-01 00:00:00,2020-02-01 00:00:00,2020-03-01 00:00:00
13,Panther,Volume,Net Migrations(Norm),Segment 1 - Sandesh Brand 2Sandesh Brand 2Mobi...,1.47449,1.46596,0.511313,0.688626,0.473906,0.331906,...,0.676655,-0.121625,0.761047,0.567425,x,x,x,x,x,x


In [10]:
df3 = df3.iloc[:, 10:46]
df3 = df3.melt(var_name="Month", value_name="Value")

In [11]:
raw_seq = np.asarray(df3.Value)
# choose a number of time steps
n_steps = 8
n_neurons = 90
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(n_neurons, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(Dropout(0.2))
model.add(LSTM(n_neurons, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(Dropout(0.2))
model.add(LSTM(n_neurons, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mape')
# fit model
model.fit(X, y, epochs=200, verbose=0)

In [12]:
# vary par = n_steps , lstm_units , droput,dense , activation , loss , optimizer

In [13]:
fc = []
for arr in X:
    x_input = []
    for i in arr:
        x_input.append(i[0])
    x_input = np.asarray(x_input)
    x_input = x_input.reshape((1, n_steps, n_features))
    yhat = model.predict(x_input, verbose=0)
    fc.append(yhat[0][0])
actual = np.asarray(df3.Value)[n_steps:]
print(fc,actual,len(fc),len(actual))

[0.0004084017, -0.02558022, -0.045276217, -0.05936766, -0.073059276, -0.07220468, -0.066082895, -0.06792468, -0.059003133, -0.0170281, 0.073265135, 0.17822811, 0.29990944, 0.4840525, 0.64152396, 0.721509, 0.6659082, 0.49642727, 0.36604685, 0.20453039, 0.32504514, 0.3356592, 0.70952666, 0.29694295, 0.9152114, 0.19895315, 0.45101395, 0.40592825] [-1.5402751843063527 -1.2909902006652985 -1.384509477241324
 -1.582171820212413 -0.5794955444748914 -1.350842537673955
 -1.4379276880215504 -0.9199057112116266 -0.4538056367567121
 -0.043966759089935276 -0.3079155652981111 0.39684570297882166
 0.969333306466623 1.011080311530161 1.3517897399519394 0.8053379030629034
 1.407602044212512 0.3863715440023059 1.061206643774911 0.951976128734113
 1.0275397042075418 0.22177761722850056 0.8831459411741572
 1.2907403562031097 0.6766553784942915 -0.12162516635866727
 0.7610471736764977 0.5674248634534932] 28 28


In [14]:
forecast_accuracy(fc,actual)

{'mape': 0.7435642981205984,
 'me': 0.1617185720436874,
 'mae': 0.6427831732249398,
 'mpe': -0.6613578350158612,
 'rmse': 0.7924649010849049}

In [15]:
df4 = df2.iloc[0:1,:]
for i in range(0,6):
    x_input = df4.iloc[:,i+46-n_steps:i+46].values.tolist()[0]
    x_input = np.asarray(x_input)
    x_input = x_input.reshape((1, n_steps, n_features))
    yhat = model.predict(x_input, verbose=0)
    df4.iloc[:,i+46] = yhat[0][0]
df4

/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:576: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value
/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


1,Generic Product,Generic Group variable,Generic_Variable,Generic_LookupKey,2016-04-01 00:00:00,2016-05-01 00:00:00,2016-06-01 00:00:00,2016-07-01 00:00:00,2016-08-01 00:00:00,2016-09-01 00:00:00,...,2019-06-01 00:00:00,2019-07-01 00:00:00,2019-08-01 00:00:00,2019-09-01 00:00:00,2019-10-01 00:00:00,2019-11-01 00:00:00,2019-12-01 00:00:00,2020-01-01 00:00:00,2020-02-01 00:00:00,2020-03-01 00:00:00
13,Panther,Volume,Net Migrations(Norm),Segment 1 - Sandesh Brand 2Sandesh Brand 2Mobi...,1.47449,1.46596,0.511313,0.688626,0.473906,0.331906,...,0.676655,-0.121625,0.761047,0.567425,0.490108,0.208511,0.338384,0.826535,0.390904,0.323646


## Net Migrations Leopard

In [16]:
df3 = df2.iloc[1:2,:]
df3.head()

1,Generic Product,Generic Group variable,Generic_Variable,Generic_LookupKey,2016-04-01 00:00:00,2016-05-01 00:00:00,2016-06-01 00:00:00,2016-07-01 00:00:00,2016-08-01 00:00:00,2016-09-01 00:00:00,...,2019-06-01 00:00:00,2019-07-01 00:00:00,2019-08-01 00:00:00,2019-09-01 00:00:00,2019-10-01 00:00:00,2019-11-01 00:00:00,2019-12-01 00:00:00,2020-01-01 00:00:00,2020-02-01 00:00:00,2020-03-01 00:00:00
16,Leopard,Volume,Net Migrations(Norm),Segment 1 - Sandesh Brand 2Sandesh Brand 2Mobi...,-1.51781,-1.4995,0.549954,0.169296,0.630261,0.935109,...,0.161265,2.13523,-0.154505,0.146809,x,x,x,x,x,x


In [17]:
df3 = df3.iloc[:, 10:46]
df3 = df3.melt(var_name="Month", value_name="Value")

In [18]:
raw_seq = np.asarray(df3.Value)
# choose a number of time steps
n_steps = 2
n_neurons = 100
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(n_neurons, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(Dropout(0.2))
model.add(LSTM(n_neurons, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(Dropout(0.2))
model.add(LSTM(n_neurons, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mape')
# fit model
model.fit(X, y, epochs=200, verbose=0)

In [19]:
fc = []
for arr in X:
    x_input = []
    for i in arr:
        x_input.append(i[0])
    x_input = np.asarray(x_input)
    x_input = x_input.reshape((1, n_steps, n_features))
    yhat = model.predict(x_input, verbose=0)
    fc.append(yhat[0][0])
actual = np.asarray(df3.Value)[n_steps:]
print(fc,actual,len(fc),len(actual))

[0.08313883, 0.19611256, 0.14524284, 0.079430655, 0.033587474, 0.038915724, 0.034651246, 0.013747698, 0.019796764, 0.03339469, 0.02226373, 0.01346191, 0.019741213, 0.05991279, 0.07307507, 0.07657296, 0.055308223, 0.039612897, 0.060977917, 0.015444675, 0.0052013644, 0.005015247, 0.004880378, 0.0050093085, 0.004897392, 0.006761784, 0.0051231068, 0.005064161, 0.005149228, 0.009334337, 0.0059750434, 0.0076757907, 0.06965397, 0.16176629] [1.4182397618363596 0.9739779545054541 0.32509230677701645
 0.37231319302260063 0.4664337349950922 0.027954077000380564
 0.06007712886812493 0.37006457939185883 0.23996621932749376
 0.0665017392416736 -0.06841507860285254 0.5772582639388103
 0.710247698671272 0.8021196270130211 0.6704151143552689
 0.28622341401704626 0.7314489129039834 0.3266984593704037
 -1.0963527383706744 -1.6617184512429768 -2.234793696563537
 -1.6745676719900742 -2.090561193677364 -0.2508740132116403
 -1.2987279651374635 -1.4898601237505438 -1.5155585652447394
 0.004825479655604475 -0.

In [20]:
forecast_accuracy(fc,actual)

{'mape': 0.8681208395571667,
 'me': 0.15712883498649705,
 'mae': 0.7279012157862393,
 'mpe': -0.8361518322547773,
 'rmse': 0.9893959790462984}

In [21]:
df4 = df2.iloc[1:2,:]
for i in range(0,6):
    x_input = df4.iloc[:,i+46-n_steps:i+46].values.tolist()[0]
    x_input = np.asarray(x_input)
    x_input = x_input.reshape((1, n_steps, n_features))
    yhat = model.predict(x_input, verbose=0)
    df4.iloc[:,i+46] = yhat[0][0]
df4

1,Generic Product,Generic Group variable,Generic_Variable,Generic_LookupKey,2016-04-01 00:00:00,2016-05-01 00:00:00,2016-06-01 00:00:00,2016-07-01 00:00:00,2016-08-01 00:00:00,2016-09-01 00:00:00,...,2019-06-01 00:00:00,2019-07-01 00:00:00,2019-08-01 00:00:00,2019-09-01 00:00:00,2019-10-01 00:00:00,2019-11-01 00:00:00,2019-12-01 00:00:00,2020-01-01 00:00:00,2020-02-01 00:00:00,2020-03-01 00:00:00
16,Leopard,Volume,Net Migrations(Norm),Segment 1 - Sandesh Brand 2Sandesh Brand 2Mobi...,-1.51781,-1.4995,0.549954,0.169296,0.630261,0.935109,...,0.161265,2.13523,-0.154505,0.146809,0.011253,0.0171608,0.0127894,0.0129035,0.0127837,0.0127855


## Net Migrations Lion

In [22]:
df3 = df2.iloc[2:3,:]
df3.head()

1,Generic Product,Generic Group variable,Generic_Variable,Generic_LookupKey,2016-04-01 00:00:00,2016-05-01 00:00:00,2016-06-01 00:00:00,2016-07-01 00:00:00,2016-08-01 00:00:00,2016-09-01 00:00:00,...,2019-06-01 00:00:00,2019-07-01 00:00:00,2019-08-01 00:00:00,2019-09-01 00:00:00,2019-10-01 00:00:00,2019-11-01 00:00:00,2019-12-01 00:00:00,2020-01-01 00:00:00,2020-02-01 00:00:00,2020-03-01 00:00:00
19,Lion,Volume,Net Migrations(Norm),Segment 1 - Sandesh Brand 2Sandesh Brand 2Mobi...,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.11952,-0.989891,-0.92848,-0.430021,x,x,x,x,x,x


In [23]:
df3 = df3.iloc[:, 10:46]
df3 = df3.melt(var_name="Month", value_name="Value")

In [24]:
raw_seq = np.asarray(df3.Value)
# choose a number of time steps
n_steps = 4
n_neurons = 100
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(n_neurons, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(Dropout(0.2))
model.add(LSTM(n_neurons, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(Dropout(0.2))
model.add(LSTM(n_neurons, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mape')
# fit model
model.fit(X, y, epochs=200, verbose=0)

In [25]:
fc = []
for arr in X:
    x_input = []
    for i in arr:
        x_input.append(i[0])
    x_input = np.asarray(x_input)
    x_input = x_input.reshape((1, n_steps, n_features))
    yhat = model.predict(x_input, verbose=0)
    fc.append(yhat[0][0])
actual = np.asarray(df3.Value)[n_steps:]
print(fc,actual,len(fc),len(actual))

[0.8553029, 0.18692484, 0.23599866, 0.15038408, 0.17960852, 0.5155235, 0.41825482, 0.6388698, 0.81518745, 0.4744392, 0.57449627, 0.5422465, 0.10238871, 0.07721877, -0.11117233, -0.37033027, -0.49157092, -0.52573925, -0.4943817, -0.34765542, -0.3148027, -0.40793878, -0.41498852, -0.5114052, -0.41607013, -0.451932, -0.42503634, -0.5767064, -0.695563, -0.72502303, -0.58183146, -0.41459754] [0.8919699204845989 1.5522300967634919 0.1555895233815728
 1.3295692575895184 1.7986095554696226 1.3148600361742266
 1.5042412618961039 1.8464145250693207 0.6915567787012535
 1.269813045589897 1.3007024105620086 0.6115753872556065
 0.11421983815106573 -0.16948426989636697 -0.09998319870911494
 -0.7343183722435567 -0.6232637505581067 -0.3509592891075245
 -0.44160486607925775 -0.09078993532455802 -0.592742116121377
 -0.3908580521965023 -0.6203219062750485 -0.36750716319972737
 -0.5311472514448442 -0.5600140984723536 -1.0266641478724734
 -1.4903723529895347 -1.1195161080565004 -0.9898910943342448
 -0.92848

In [26]:
forecast_accuracy(fc,actual)

{'mape': 0.5127935684436749,
 'me': -0.16666603850055256,
 'mae': 0.43035623476443613,
 'mpe': -0.2197008536888383,
 'rmse': 0.6220607969454875}

In [27]:
df4 = df2.iloc[2:3,:]
for i in range(0,6):
    x_input = df4.iloc[:,i+46-n_steps:i+46].values.tolist()[0]
    x_input = np.asarray(x_input)
    x_input = x_input.reshape((1, n_steps, n_features))
    yhat = model.predict(x_input, verbose=0)
    df4.iloc[:,i+46] = yhat[0][0]
df4

1,Generic Product,Generic Group variable,Generic_Variable,Generic_LookupKey,2016-04-01 00:00:00,2016-05-01 00:00:00,2016-06-01 00:00:00,2016-07-01 00:00:00,2016-08-01 00:00:00,2016-09-01 00:00:00,...,2019-06-01 00:00:00,2019-07-01 00:00:00,2019-08-01 00:00:00,2019-09-01 00:00:00,2019-10-01 00:00:00,2019-11-01 00:00:00,2019-12-01 00:00:00,2020-01-01 00:00:00,2020-02-01 00:00:00,2020-03-01 00:00:00
19,Lion,Volume,Net Migrations(Norm),Segment 1 - Sandesh Brand 2Sandesh Brand 2Mobi...,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.11952,-0.989891,-0.92848,-0.430021,-0.390468,-0.289382,-0.246069,-0.366253,-0.383052,-0.431489
